In [ ]:
print("OK!")

In [ ]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
import sys
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage
import json
import ast


In [ ]:
# Load environment variables
load_dotenv()

In [ ]:
# Check for ANTHROPIC_API_KEY
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    print("Error: ANTHROPIC_API_KEY is not set.")
    print("Please set your Anthropic API key in the .env file or as an environment variable.")
    print("Example .env file content:")
    print("ANTHROPIC_API_KEY=your_api_key_here")
    sys.exit(1)

os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Java Code Evaluation"

class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    java_code: str
    extracted_classes: List[str]
    rubric: dict
    initial_evaluation: dict
    review_evaluation: dict
    marks: dict
    total_marks: float

graph_builder = StateGraph(State)

llm = ChatAnthropic(model="claude-3-haiku-20240307")

def safe_eval(response_content):
    try:
        return ast.literal_eval(response_content)
    except:
        try:
            return json.loads(response_content)
        except:
            return {}

def class_extraction(state: State):
    prompt = f"""
    Extract the class names from the following Java code:
    {state['java_code']}
    Return only a list of class names.
    """
    response = llm.invoke(prompt)
    extracted_classes = [class_name.strip() for class_name in response.content.split('\n') if class_name.strip()]
    return {"extracted_classes": extracted_classes}

def rubric_extraction(state: State):
    prompt = """
    Create a rubric for evaluating Java code. The rubric should include criteria for:
    1. Code structure and organization
    2. Proper use of object-oriented principles
    3. Naming conventions and readability
    4. Error handling and input validation
    5. Comments and documentation

    For each criterion, provide a description and the maximum points available.
    Return the rubric as a JSON object.
    """
    response = llm.invoke(prompt)
    rubric = safe_eval(response.content)
    return {"rubric": rubric}

def initial_evaluation(state: State):
    prompt = f"""
    Evaluate the following Java code based on the given rubric:
    
    Code:
    {state['java_code']}
    
    Rubric:
    {state['rubric']}
    
    Provide an initial evaluation for each criterion in the rubric.
    Return the evaluation as a JSON object with the same structure as the rubric.
    """
    response = llm.invoke(prompt)
    initial_evaluation = safe_eval(response.content)
    return {"initial_evaluation": initial_evaluation}

def review_evaluation(state: State):
    prompt = f"""
    Review and refine the initial evaluation of the Java code:
    
    Code:
    {state['java_code']}
    
    Rubric:
    {state['rubric']}
    
    Initial Evaluation:
    {state['initial_evaluation']}
    
    Provide a refined evaluation, considering any aspects that might have been overlooked.
    Return the refined evaluation as a JSON object with the same structure as the initial evaluation.
    """
    response = llm.invoke(prompt)
    review_evaluation = safe_eval(response.content)
    return {"review_evaluation": review_evaluation}

def marks_extraction(state: State):
    prompt = f"""
    Based on the reviewed evaluation, assign marks for each criterion:
    
    Rubric:
    {state['rubric']}
    
    Reviewed Evaluation:
    {state['review_evaluation']}
    
    Assign marks for each criterion, ensuring they do not exceed the maximum points specified in the rubric.
    Return the marks as a JSON object with the same structure as the rubric and evaluations.
    """
    response = llm.invoke(prompt)
    marks = safe_eval(response.content)
    return {"marks": marks}

def total_marks_calculation(state: State):
    total = sum(state['marks'].values())
    return {"total_marks": total}

In [ ]:
# Add nodes to the graph
graph_builder.add_node("extract_classes", class_extraction)
graph_builder.add_node("extract_rubric", rubric_extraction)
graph_builder.add_node("evaluate_initially", initial_evaluation)
graph_builder.add_node("review_eval", review_evaluation)
graph_builder.add_node("extract_marks", marks_extraction)
graph_builder.add_node("calculate_total_marks", total_marks_calculation)

In [ ]:
# Add edges to the graph
graph_builder.add_edge(START, "extract_classes")
graph_builder.add_edge("extract_classes", "extract_rubric")
graph_builder.add_edge("extract_rubric", "evaluate_initially")
graph_builder.add_edge("evaluate_initially", "review_eval")
graph_builder.add_edge("review_eval", "extract_marks")
graph_builder.add_edge("extract_marks", "calculate_total_marks")
graph_builder.add_edge("calculate_total_marks", END)


In [ ]:
# Compile the graph
graph = graph_builder.compile()

In [ ]:
# Main execution loop
while True:
    print("\nEnter Java code to evaluate (or 'quit' to exit):")
    java_code = ""
    while True:
        line = input()
        if line.lower() == "quit":
            print("Goodbye!")
            sys.exit(0)
        if line.strip() == "":
            break
        java_code += line + "\n"
    
    if java_code.strip():
        initial_state = State(
            messages=[],
            java_code=java_code,
            extracted_classes=[],
            rubric={},
            initial_evaluation={},
            review_evaluation={},
            marks={},
            total_marks=0.0
        )

        try:
            for event in graph.stream(initial_state):
                for key, value in event.items():
                    if key == "messages":
                        for message in value:
                            if isinstance(message, BaseMessage):
                                print(f"{message.type.capitalize()}: {message.content}")
                    else:
                        print(f"{key.replace('_', ' ').capitalize()}:")
                        print(json.dumps(value, indent=2))
                        print()

            print(f"Total Marks: {initial_state['total_marks']}")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please try again with a different Java code snippet.")
    else:
        print("No code entered. Please try again.")